# Flights Data Exploration Challenge

In this challge, you'll explore a real-world dataset containing flights data from the US Department of Transportation.

Let's start by loading and viewing the data.

In [ ]:
import pandas as pd

df_flights = pd.read_csv('data/flights.csv')
df_flights.head()

In [ ]:
The dataset contains observations of US domestic flights in 2013, and consists of the following fields:

- **Year**: The year of the flight (all records are from 2013)
- **Month**: The month of the flight
- **DayofMonth**: The day of the month on which the flight departed
- **DayOfWeek**: The day of the week on which the flight departed - from 1 (Monday) to 7 (Sunday)
- **Carrier**: The two-letter abbreviation for the airline.
- **OriginAirportID**: A unique numeric identifier for the departure aiport
- **OriginAirportName**: The full name of the departure airport
- **OriginCity**: The departure airport city
- **OriginState**: The departure airport state
- **DestAirportID**: A unique numeric identifier for the destination aiport
- **DestAirportName**: The full name of the destination airport
- **DestCity**: The destination airport city
- **DestState**: The destination airport state
- **CRSDepTime**: The scheduled departure time
- **DepDelay**: The number of minutes departure was delayed (flight that left ahead of schedule have a negative value)
- **DelDelay15**: A binary indicator that departure was delayed by more than 15 minutes (and therefore considered "late")
- **CRSArrTime**: The scheduled arrival time
- **ArrDelay**: The number of minutes arrival was delayed (flight that arrived ahead of schedule have a negative value)
- **ArrDelay15**: A binary indicator that arrival was delayed by more than 15 minutes (and therefore considered "late")
- **Cancelled**: A binary indicator that the flight was cancelled

Your challenge is to explore the flight data to analyze possible factors that affect delays in departure or arrival of a flight.

1. Start by cleaning the data.
    - Identify any null or missing data, and impute appropriate replacement values.
    - Identify and eliminate any outliers in the **DepDelay** and **ArrDelay** columns.
2. Explore the cleaned data.
    - View summary statistics for the numeric fields in the dataset.
    - Determine the distribution of the **DepDelay** and **ArrDelay** columns.
    - Use statistics, aggregate functions, and visualizations to answer the following questions:
        - *What are the average (mean) departure and arrival delays?*
        - *How do the carriers compare in terms of arrival delay performance?*
        - *Is there a noticable difference in arrival delays for different days of the week?*
        - *Which departure airport has the highest average departure delay?*
        - *Do **late** departures tend to result in longer arrival delays than on-time departures?*
        - *Which route (from origin airport to destination airport) has the most **late** arrivals?*
        - *Which route has the highest average arrival delay?*
        
Add markdown and code cells as required to create your solution.

> **Note**: There is no single "correct" solution. A sample solution is provided in [01 - Flights Challenge.ipynb](01%20-%20Flights%20Solution.ipynb).

In [ ]:
# Your code to explore the data
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import gridspec
import numpy as np

# Step 1: Cleaning Data
## Missing Values

Let's see which columns have missing values:



In [ ]:
df_flights.isnull().sum()

Only **DepDel15** has missing values. 

Are there columns with empty stings?  

In [ ]:
for var in list(df_flights.columns):
    rowcount = sum(df_flights[var] == '')
    print('{}: {}'.format(var, rowcount))


Nope! 
These are the rows with missing values for **DepDel15**:

In [ ]:
df_flights[df_flights.isnull().any(axis=1)]

Are all flights with missing **DepDel15** cancelled?

In [ ]:
df_flights[df_flights.isnull().any(axis=1)].groupby('Cancelled')['DepDelay'].describe()


All rows with missing **DepDel15** are cancelled flights with `0` DepDelay.
So, I'll replace all missing **DepDel15** with `0`. 

In [ ]:
# this doesn't work:
#df_flights[df_flights.isnull().any(axis=1)]['DepDel15'] = 0

# this works:
#df_flights.DepDel15 = df_flights.DepDel15.fillna(0)
#df_flights.DepDel15[df_flights.isnull().any(axis=1)] = 0
#df_flights['DepDel15'][df_flights.isnull().any(axis=1)] = 0

df_flights.DepDel15 = pd.np.where(df_flights.isnull().any(axis=1), 0, df_flights.DepDel15)

Check again for missing valeus

In [ ]:
df_flights.isnull().sum()

In [ ]:
Are there cancelled flights with delay??

In [ ]:
df_flights.query('Cancelled == 1').query('DepDelay != 0')

Yes, there are.  

And cancelled flights with arrivaldalay??

In [ ]:
df_flights.query('Cancelled == 1').query('ArrDelay != 0')

In [ ]:
No, that would be odd.

## Outliers

In [ ]:
This functions produces plot and stats to see the distribution of a variable:

In [ ]:
def print_stats(var):
   
# the stats
    mean_var = var.mean()
    std_var = var.std()
    min_var = var.min()
    q01_var = var.quantile(0.01)
    q25_var = var.quantile(0.25)
    median_var = var.median()
    q75_var = var.quantile(0.75)
    q99_var = var.quantile(0.99)
    max_var = var.max()
    mode_var = var.mode()[0]
    
    stats = 'Mean          : {:.2f}\nStd.Dev      : {:.2f}\nMode          : {:.2f}\nMinimum    : {:.2f}\n01-quantile: {:.2f}\n25-quantile: {:.2f}\nMedian       : {:.2f}\n75-quantile: {:.2f}\n99-quantile: {:.2f}\nMaximum   : {:.2f}'.format(
                                mean_var
                               ,std_var
                               ,mode_var
                               ,min_var
                               ,q01_var
                               ,q25_var
                               ,median_var
                               ,q75_var
                               ,q99_var 
                               ,max_var)
   
    # set up figure
    fig, ax = plt.subplots(2, 1
                    ,figsize = (15, 5)
                    ,sharex = True
                    ,gridspec_kw = {'hspace': 0
                                   ,'height_ratios': [5, 1]})
   

 
    fig.suptitle('Distribution ' + var.name)
    
    # histogram
    ax[0].hist(var,  bins = np.arange(min_var, max_var, (q99_var - q01_var)/15))
    ax[0].set_ylabel('Frequency')
    # stat-lines
    ax[0].axvline(x = q01_var, color = 'gray', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x = q99_var, color = 'gray', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x = mean_var, color = 'black', linestyle ='solid', linewidth = 2)
    ax[0].axvline(x = median_var, color = 'black', linestyle='dashed', linewidth = 2)

    ax[0].annotate(stats, xy = (.85, .5), xycoords='axes fraction')
    
    # boxplot
    ax[1].boxplot(var, vert = False)
    ax[1].set_xlabel('Value')

    ax[0].label_outer()
    fig.show()

### DepDelay

In [ ]:
print_stats(df_flights['DepDelay'])

**DepDelay** has a very fat tail on the right.

### ArrDelay

In [ ]:
print_stats(df_flights['ArrDelay'])

**ArrDelay** has also a big fat tail on the right.  

Now let's make a scatterplot of these two variables. Because some cancelled flight have departdelays, I want to be able to indentify them.

In [ ]:
# the quantiles
qdep_01 = df_flights['DepDelay'].quantile(0.01)
qdep_99 = df_flights['DepDelay'].quantile(0.99)
qarr_01 = df_flights['ArrDelay'].quantile(0.01)
qarr_99 = df_flights['ArrDelay'].quantile(0.99)


In [ ]:
# scatterplot
plt.scatter(x = df_flights['DepDelay'], y = df_flights['ArrDelay'], c = df_flights['Cancelled'])
# quantile lines
plt.axvline(x = qdep_01, color = 'gray', linestyle = 'dashed', linewidth = 2)
plt.axvline(x = qdep_99, color = 'gray', linestyle = 'dashed', linewidth = 2)
plt.axhline(y = qarr_01, color = 'gray', linestyle = 'dashed', linewidth = 2)
plt.axhline(y = qarr_99, color = 'gray', linestyle = 'dashed', linewidth = 2)

In [ ]:
To clean the data, I will remove the cancelled flights and everything outside the 1% and 99% quantiles of **DepDelay** and **ArrDelay**.

In [ ]:
# remove cancelled flights
flights = df_flights[df_flights['Cancelled'] == 0]

# remove outside quantile area
flights = flights[flights['DepDelay'] > qdep_01][flights['DepDelay'] < qdep_99][flights['ArrDelay'] > qarr_01][flights['ArrDelay'] < qarr_99]

# same scatterplot for cleaned data
plt.scatter(x =  flights['DepDelay'], y =  flights['ArrDelay'])


## Step 2: Explore

Now, let's see de distributions and statistics of **DepDelay** and **ArrDelay** of the cleaned data.

In [ ]:
print_stats(flights['DepDelay'])

In [ ]:
print_stats(flights['ArrDelay'])

In [ ]:
- Use statistics, aggregate functions, and visualizations to answer the following questions:
        - *What are the average (mean) departure and arrival delays?*
        - *How do the carriers compare in terms of arrival delay performance?*
        - *Is there a noticable difference in arrival delays for different days of the week?*
        - *Which departure airport has the highest average departure delay?*
        - *Do **late** departures tend to result in longer arrival delays than on-time departures?*
        - *Which route (from origin airport to destination airport) has the most **late** arrivals?*
        - *Which route has the highest average arrival delay?*

What are the average (mean) departure and arrival delays?

In [ ]:
flights[['DepDelay','ArrDelay']].mean()

How do the carriers compare in terms of arrival delay performance?

In [ ]:
flights.groupby('Carrier')[['DepDelay','ArrDelay']].describe()

In [ ]:
flights.boxplot(column ='DepDelay', vert = False, by = 'Carrier', figsize=(15,8))

In [ ]:
flights.boxplot(column ='ArrDelay', vert = False, by = 'Carrier', figsize=(15, 8))

Is there a noticable difference in arrival delays for different days of the week?

In [ ]:
flights.groupby('DayOfWeek')['ArrDelay'].describe()

In [ ]:
flights.boxplot(column ='ArrDelay', vert = False, by = 'DayOfWeek', figsize=(15,8))

Which departure airport has the highest average departure delay?

In [ ]:
grouped = flights.groupby('OriginAirportName')['DepDelay'].mean()
depdelay_means = pd.DataFrame(grouped).sort_values('DepDelay', ascending = False)
depdelay_means 

That would be *Chicago Midway International*.

In [ ]:
# barchart

#depdelay_means.plot.bar(x='OriginAirportName', y='DepDelay', color='teal', figsize=(16,6))
depdelay_means.plot(kind = "bar", figsize=(12,12))

Do **late** departures tend to result in longer arrival delays than on-time 

In [ ]:
plt.scatter(x =  flights['DepDelay'], y =  flights['ArrDelay'])

In [ ]:
corr = np.corrcoef( flights['DepDelay'],   flights['ArrDelay'])[0,1]
corr

In [ ]:
flights.boxplot(column='ArrDelay', by='DepDel15', figsize=(12,12))

In [ ]:
Not always, but yes. There is a positive correlation.

Which route (from origin airport to destination airport) has the most **late** arrivals?*  

In [ ]:
routes = flights.groupby(['OriginAirportName', 'DestAirportName'])['ArrDel15'].sum()
route_lates = pd.DataFrame(routes).sort_values('ArrDel15', ascending = False)
route_lates


From *Los Angeles International* to *San Francisco International*

Which route has the highest average arrival delay?

In [ ]:
routesd = flights.groupby(['OriginAirportName', 'DestAirportName'])['ArrDelay'].mean()
route_latesd = pd.DataFrame(routesd).sort_values('ArrDelay', ascending = False)
route_latesd

From *Pittsburgh International* to *Raleigh-Durham International*